In [1]:
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass
from tqdm.notebook import tqdm
import pandas as pd
import sqlite3

In [2]:

PLAYBACK_URL = "https://playback.fm/charts/top-100-songs/"
FIRST_YEAR = 1985
LAST_YEAR = 2021

In [3]:
@dataclass
class Track:
    year: int
    artist: str
    track: str
    rank: int
    link: str = None
    def __str__(self):
        return f"{self.year} - {self.rank}: {self.track} - {self.artist} ({self.link})"

    # Originally defined for pandas function linked below, but not necessary anymore!
    # https://stackoverflow.com/questions/34997174/how-to-convert-list-of-model-objects-to-pandas-dataframe
    def to_dict(self):
        return {
            'year': self.year,
            'artist': self.artist,
            'track': self.track,
            'link': self.link,
        }

In [4]:
def get_songs_from(year: int):
    site = PLAYBACK_URL + "/" + str(year)
    page = requests.get(site)
    soup = BeautifulSoup(page.content, 'html.parser')
    soup_tracks = soup.find_all("tr", itemprop="track")
    tracks = []
    for soup in tqdm(soup_tracks):
        artist = soup.find("a", itemprop="byArtist", class_="artist").text.strip() or ""
        link = soup.find("a")["href"].strip()
        track = soup.find("span", class_="song", itemprop="name").text.strip() or ""
        rank = int(soup.find("td").text.strip())
        track = Track(year=year,artist=artist,track=track, link=link, rank=rank)
        # print(track)
        tracks.append(track)
    return tracks

In [5]:
all_tracks = []
for year in tqdm(range(FIRST_YEAR, LAST_YEAR + 1)):
    all_tracks = all_tracks + get_songs_from(year)

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

## Convert to Pandas, SQL Export

In [6]:
tracks_df = pd.DataFrame([vars(track) for track in all_tracks])

In [10]:
# Drop Duplicates
tracks_df.drop_duplicates(subset=["artist", "track"], inplace=True, ignore_index=True)

In [12]:
tracks_db = sqlite3.connect('22-04-29-playback-fm-top-pop.db')
tracks_df.to_sql("tracks", tracks_db, if_exists="replace")

3595

In [13]:
tracks_db.close()